In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("simulated_data.csv")
df.head()

,user_id,account_id,user_role,plan_type,country,timezone,session_length_seconds,pages_visited,last_seen_days_ago,conversation_open,previous_messages_sent,previous_messages_clicked,previous_messages_dismissed,message_type,send_message,clicked,replied,conversation_resolved
0,0,1339,end_user,pro,US,-1,311,5,1,0,1,1,0,bug_help,1,1,1,1
1,1,11609,end_user,pro,APAC,8,353,8,7,1,0,0,0,bug_help,1,1,1,0
2,2,9818,end_user,pro,LATAM,-3,216,4,0,0,2,1,1,none,0,1,1,1
3,3,6583,admin,pro,LATAM,-5,348,5,10,0,2,0,2,onboarding_tip,0,0,0,0
4,4,6495,end_user,pro,EU,6,301,4,13,0,1,1,0,onboarding_tip,0,0,0,1


In [3]:
df["reward"] = (
    1.5 * df["replied"]
    + 0.5 * df["clicked"]
    - 1.0 * df["previous_messages_dismissed"]
)
df.head()

,user_id,account_id,user_role,plan_type,country,timezone,session_length_seconds,pages_visited,last_seen_days_ago,conversation_open,previous_messages_sent,previous_messages_clicked,previous_messages_dismissed,message_type,send_message,clicked,replied,conversation_resolved,reward
0,0,1339,end_user,pro,US,-1,311,5,1,0,1,1,0,bug_help,1,1,1,1,2.0
1,1,11609,end_user,pro,APAC,8,353,8,7,1,0,0,0,bug_help,1,1,1,0,2.0
2,2,9818,end_user,pro,LATAM,-3,216,4,0,0,2,1,1,none,0,1,1,1,1.0
3,3,6583,admin,pro,LATAM,-5,348,5,10,0,2,0,2,onboarding_tip,0,0,0,0,-2.0
4,4,6495,end_user,pro,EU,6,301,4,13,0,1,1,0,onboarding_tip,0,0,0,1,0.0


“I’d optimize expected reward rather than raw click-through rate.”

# Exploratory Data Analysis

In [4]:
df[['clicked', 'replied', 'conversation_resolved']].describe()

,clicked,replied,conversation_resolved
count,100000.000000,100000.000000,100000.00000
mean,0.535840,0.542900,0.51682
std,0.498716,0.498159,0.49972
min,0.000000,0.000000,0.00000
25%,0.000000,0.000000,0.00000
50%,1.000000,1.000000,1.00000
75%,1.000000,1.000000,1.00000
max,1.000000,1.000000,1.00000


In [5]:
df.groupby('message_type')['clicked'].mean()

message_type
bug_help          0.533267
none              0.531322
onboarding_tip    0.538431
pricing_help      0.538353
Name: clicked, dtype: float64

# Feature engineering

In [6]:
df["engagement_rate"] = (
    df["previous_messages_clicked"] /
    (df["previous_messages_sent"] + 1)
)

df["fatigue_score"] = (
    df["previous_messages_sent"] -
    df["previous_messages_clicked"]
)

features = [
    "session_length_seconds",
    "pages_visited",
    "last_seen_days_ago",
    "conversation_open",
    "engagement_rate",
    "fatigue_score"
]

target = "clicked"

# Baseline model

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [8]:
X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

preds = model.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, preds)

0.9038676715150068

“This gives me a strong, interpretable baseline.”

# Contextual bandit formulation

In [9]:
ACTIONS = ["none", "onboarding_tip", "pricing_help", "bug_help"]

In [10]:
df.head()

,user_id,account_id,user_role,plan_type,country,timezone,session_length_seconds,pages_visited,last_seen_days_ago,conversation_open,...,previous_messages_clicked,previous_messages_dismissed,message_type,send_message,clicked,replied,conversation_resolved,reward,engagement_rate,fatigue_score
0,0,1339,end_user,pro,US,-1,311,5,1,0,...,1,0,bug_help,1,1,1,1,2.0,0.500000,0
1,1,11609,end_user,pro,APAC,8,353,8,7,1,...,0,0,bug_help,1,1,1,0,2.0,0.000000,0
2,2,9818,end_user,pro,LATAM,-3,216,4,0,0,...,1,1,none,0,1,1,1,1.0,0.333333,1
3,3,6583,admin,pro,LATAM,-5,348,5,10,0,...,0,2,onboarding_tip,0,0,0,0,-2.0,0.000000,2
4,4,6495,end_user,pro,EU,6,301,4,13,0,...,1,0,onboarding_tip,0,0,0,1,0.0,0.500000,0


In [11]:
from sklearn.linear_model import SGDRegressor
from collections import defaultdict

SGD Regressor is important for reasons like:

- It can handle big datasets and is computationally efficient. It is therefore appropriate for big data applications.
- It has the ability to learn online, which allows it to update the model whenever fresh data becomes available. This is crucial for applications that use real-time data streams.
- It is appropriate for distributed computing systems since it may be parallelized.
- To avoid overfitting and enhance generalization, SGD can be expanded to incorporate regularization strategies as L1 (Lasso) and L2 (Ridge) regularization.
- It may be applied to various regression algorithms, such as support vector machines (SVM) and neural networks, and is not just restricted to linear regression.

In [12]:
df_sample = df.sample(1000)
X_train = df_sample[features].values
y_train = df_sample[target].values

In [13]:
models = {
    action: SGDRegressor(learning_rate="constant", eta0=0.01)
    for action in ACTIONS
}

# Initialize
for m in models.values():
    m.partial_fit(X_train[:10], np.zeros(10))

In [14]:
import random

from tqdm import tqdm
tqdm.pandas()

In [15]:
def select_action(context, epsilon=0.1):
    if random.random() < epsilon:
        return random.choice(ACTIONS)
    scores = {
        a: models[a].predict(context.reshape(1, -1))[0]
        for a in ACTIONS
    }
    return max(scores, key=scores.get)

In [16]:
total_reward = 0

with tqdm(total=len(df_sample)) as pbar:

    for _, row in df_sample.iterrows():
    #for _, row in tqdm(df.iterrows(), total=len(df)):

        context = row[features].values
        action = select_action(context)

        reward = row["reward"] if row["message_type"] == action else 0

        models[action].partial_fit(
            context.reshape(1, -1),
            [reward]
        )

        total_reward += reward

        pbar.update(1)

        # Display the completion percentage
        print(f"Completion: {pbar.n}/{pbar.total}")

total_reward

  8%|▊         | 76/1000 [00:00<00:02, 353.98it/s]

Completion: 1/1000
Completion: 2/1000
Completion: 3/1000
Completion: 4/1000
Completion: 5/1000
Completion: 6/1000
Completion: 7/1000
Completion: 8/1000
Completion: 9/1000
Completion: 10/1000
Completion: 11/1000
Completion: 12/1000
Completion: 13/1000
Completion: 14/1000
Completion: 15/1000
Completion: 16/1000
Completion: 17/1000
Completion: 18/1000
Completion: 19/1000
Completion: 20/1000
Completion: 21/1000
Completion: 22/1000
Completion: 23/1000
Completion: 24/1000
Completion: 25/1000
Completion: 26/1000
Completion: 27/1000
Completion: 28/1000
Completion: 29/1000
Completion: 30/1000
Completion: 31/1000
Completion: 32/1000
Completion: 33/1000
Completion: 34/1000
Completion: 35/1000
Completion: 36/1000
Completion: 37/1000
Completion: 38/1000
Completion: 39/1000
Completion: 40/1000
Completion: 41/1000
Completion: 42/1000
Completion: 43/1000
Completion: 44/1000
Completion: 45/1000
Completion: 46/1000
Completion: 47/1000
Completion: 48/1000
Completion: 49/1000
Completion: 50/1000
Completio

 26%|██▌       | 258/1000 [00:00<00:01, 564.84it/s]

Completion: 109/1000
Completion: 110/1000
Completion: 111/1000
Completion: 112/1000
Completion: 113/1000
Completion: 114/1000
Completion: 115/1000
Completion: 116/1000
Completion: 117/1000
Completion: 118/1000
Completion: 119/1000
Completion: 120/1000
Completion: 121/1000
Completion: 122/1000
Completion: 123/1000
Completion: 124/1000
Completion: 125/1000
Completion: 126/1000
Completion: 127/1000
Completion: 128/1000
Completion: 129/1000
Completion: 130/1000
Completion: 131/1000
Completion: 132/1000
Completion: 133/1000
Completion: 134/1000
Completion: 135/1000
Completion: 136/1000
Completion: 137/1000
Completion: 138/1000
Completion: 139/1000
Completion: 140/1000
Completion: 141/1000
Completion: 142/1000
Completion: 143/1000
Completion: 144/1000
Completion: 145/1000
Completion: 146/1000
Completion: 147/1000
Completion: 148/1000
Completion: 149/1000
Completion: 150/1000
Completion: 151/1000
Completion: 152/1000
Completion: 153/1000
Completion: 154/1000
Completion: 155/1000
Completion: 1

 38%|███▊      | 378/1000 [00:00<00:01, 582.93it/s]

...

 93%|█████████▎| 928/1000 [00:01<00:00, 646.82it/s]

Completion: 863/1000
Completion: 864/1000
Completion: 865/1000
Completion: 866/1000
Completion: 867/1000
Completion: 868/1000
Completion: 869/1000
Completion: 870/1000
Completion: 871/1000
Completion: 872/1000
Completion: 873/1000
Completion: 874/1000
Completion: 875/1000
Completion: 876/1000
Completion: 877/1000
Completion: 878/1000
Completion: 879/1000
Completion: 880/1000
Completion: 881/1000
Completion: 882/1000
Completion: 883/1000
Completion: 884/1000
Completion: 885/1000
Completion: 886/1000
Completion: 887/1000
Completion: 888/1000
Completion: 889/1000
Completion: 890/1000
Completion: 891/1000
Completion: 892/1000
Completion: 893/1000
Completion: 894/1000
Completion: 895/1000
Completion: 896/1000
Completion: 897/1000
Completion: 898/1000
Completion: 899/1000
Completion: 900/1000
Completion: 901/1000
Completion: 902/1000
Completion: 903/1000
Completion: 904/1000
Completion: 905/1000
Completion: 906/1000
Completion: 907/1000
Completion: 908/1000
Completion: 909/1000
Completion: 9

100%|██████████| 1000/1000 [00:01<00:00, 583.73it/s]

Completion: 986/1000
Completion: 987/1000
Completion: 988/1000
Completion: 989/1000
Completion: 990/1000
Completion: 991/1000
Completion: 992/1000
Completion: 993/1000
Completion: 994/1000
Completion: 995/1000
Completion: 996/1000
Completion: 997/1000
Completion: 998/1000
Completion: 999/1000
Completion: 1000/1000


-107.0

This simulates learning from interaction without needing full supervision.

## Saving model

In [17]:
import joblib

joblib.dump(models, "bandit_models.joblib")

['bandit_models.joblib']

## Loading the models

In [18]:
import joblib

models_loaded = joblib.load("bandit_models.joblib")

## Inference

In [19]:
def predict_action_sgd(context):
    # context must be 2D for sklearn
    context = context.reshape(1, -1)

    predicted_rewards = {
        action: models[action].predict(context)[0]
        for action in ACTIONS
    }

    return max(predicted_rewards, key=predicted_rewards.get)

In [23]:
new_user_features = df_sample[features].sample(1).values

In [24]:
context = new_user_features.astype(float)
best_action = predict_action_sgd(context)

In [25]:
best_action

'pricing_help'